In [1]:
import nltk
from nltk.collocations import *
from nltk.metrics.spearman import *

In [2]:
from nltk.parse import DependencyGraph

In [3]:
with open('testset2.txt', encoding = 'utf-8') as f:
    text = f.read()

In [ ]:
!C:\Users\Maria\Downloads\bin-win64\udpipe --input horizontal --output conllu \
--tokenize --tag --parse \
C:\Users\Maria\Downloads\russian-syntagrus-ud-2.4-190531.udpipe \
< testset2.txt > testset2.conllu

In [4]:
trees = []

with open('testset2.conllu', encoding = 'utf-8') as f:
    parsed_sents = f.read().split('\n\n')

    for sent in parsed_sents:
        tree = [line for line in sent.split('\n') if line and line[0] != '#']
        trees.append('\n'.join(tree))

In [6]:
verb_acc = []
verb_count = {}

for tree in trees:
    g = DependencyGraph(tree, top_relation_label='root')
    for n in g.nodes:
        if g.nodes[n]['ctag'] == 'VERB':
            try:
                verb_count[g.nodes[n]['word']] += 1
            except KeyError:
                verb_count[g.nodes[n]['word']] = 1
            if 'Case=Acc' in g.nodes[n+1]['feats'] and g.nodes[n+1]['ctag'] == 'NOUN':
                verb_acc.append([g.nodes[n]['lemma'], g.nodes[n+1]['word']])
           

AssertionError: 

In [7]:
verb_acc

[['запускать', 'механизм'],
 ['рассмотреть', 'вопрос'],
 ['терпеть', 'бедствие'],
 ['определить', 'срок'],
 ['вызвать', 'недовольство'],
 ['привести', 'обитателей'],
 ['получить', 'сроки'],
 ['собирать', 'доказательства'],
 ['обвинить', 'олигарха'],
 ['предать', 'суду'],
 ['просить', 'суд'],
 ['рассматривать', 'ходатайство'],
 ['затруднять', 'доступ'],
 ['отозвать', 'иск'],
 ['отозвать', 'иск'],
 ['иметь', 'вид'],
 ['избрать', 'меру'],
 ['оказать', 'давление'],
 ['дать', 'согласие'],
 ['возбуждать', 'дело'],
 ['признать', 'сделку'],
 ['обвинить', 'господина'],
 ['удовлетворить', 'жалобу'],
 ['вынести', 'постановление'],
 ['нанести', 'убытки'],
 ['выполнить', 'условия'],
 ['обеспечить', 'легитимность'],
 ['связывать', 'объявление'],
 ['вызвать', 'появление'],
 ['просить', 'суд'],
 ['прекратить', 'дело'],
 ['заключать', 'контракты'],
 ['зафрахтовать', 'суда'],
 ['признать', 'сделку'],
 ['принять', 'решение'],
 ['запустить', 'законодательство'],
 ['выплатить', 'компенсации'],
 ['совершить

вышло так, что у меня глагол, встретившийся >= 50 раз,
всего один, а прямого дополнения после него и вовсе не встретилось, поэтому я работала со всей выборкой глаголов, она не была такой большой

In [8]:
for verb in verb_count:
    if verb_count[verb] >=50:
        print(verb)

заявил


In [9]:
tokens = nltk.wordpunct_tokenize(text)
finder = BigramCollocationFinder.from_words(tokens)
bigram_measures = nltk.collocations.BigramAssocMeasures()

In [10]:
finder.apply_ngram_filter(lambda w1, w2: [w1, w2] not in verb_acc)

pmi = finder.score_ngrams(bigram_measures.pmi)
lklh = finder.score_ngrams(bigram_measures.likelihood_ratio)
dice = finder.score_ngrams(bigram_measures.dice)

print('PMI\n', 
      pmi)
print('likelihood_ratio\n', 
      lklh)
print('DICE\n', 
      dice)


PMI
 [(('дискредитировать', 'добросовестность'), 17.359629467980064), (('запускать', 'механизм'), 17.359629467980064), (('облагородить', 'капитализм'), 17.359629467980064), (('обидеть', 'палестинцев'), 16.359629467980064), (('информировать', 'россиян'), 15.774666967258907), (('преследовать', 'неприятности'), 15.774666967258907), (('соответствовать', 'парку'), 15.774666967258907), (('связывать', 'объявление'), 15.359629467980064), (('приобщать', 'бумаги'), 15.037701373092702), (('возмещать', 'затраты'), 14.774666967258907), (('запустить', 'законодательство'), 14.55227454592246), (('брать', 'мзду'), 14.359629467980064), (('вызвать', 'появление'), 14.359629467980064), (('опубликовать', 'опровержение'), 14.189704466537751), (('обеспечить', 'легитимность'), 14.037701373092702), (('отсрочить', 'исполнение'), 13.774666967258907), (('представить', 'перечень'), 13.774666967258907), (('купить', 'квартиру'), 13.452738872371544), (('отделить', 'завод'), 13.452738872371544), (('оказать', 'давление'

In [11]:
pmi_to_set = [' '.join(list(k[0])) for k in pmi[:50]]
lklh_to_set = [' '.join(list(k[0])) for k in lklh[:50]]
dice_to_set = [' '.join(list(k[0])) for k in dice[:50]]

best_anyway = set(pmi_to_set) & set(lklh_to_set) & set(dice_to_set)

In [12]:
with open('verb_coll.txt', encoding = 'utf-8') as f:
    verb_dict = f.read()

In [13]:
lines = verb_dict.split('\n')

In [14]:
from_dict = [k.split('\t')[2] for k in lines]

In [15]:
final = best_anyway & set(from_dict)

In [16]:
final = list(final)

Добавим в ЗС следующие коллокации:

возмещать затраты

расторгнуть договор

оспаривать решение

подать апелляцию

Некоторые похожие конструкции были определены как коллокации метриками. В целом, мне кажется, существительные из этих выражений сочетаются с небольшим количеством глаголов, в том числе как раз с теми, что вошли в выбранные коллокации. 

In [17]:
final += ['возмещать затраты', 'расторгнуть договор', 'оспаривать решение', 'подать апелляцию']

In [18]:
final

['опубликовать опровержение',
 'давать показания',
 'принимать участие',
 'рассмотреть вопрос',
 'вызвать появление',
 'дать показания',
 'оказать давление',
 'возместить ущерб',
 'возмещать затраты',
 'расторгнуть договор',
 'оспаривать решение',
 'подать апелляцию']

In [19]:
results_pmi = list(ranks_from_sequence(pmi_to_set))
results_dice = list(ranks_from_sequence(dice_to_set))
results_lklh = list(ranks_from_sequence(lklh_to_set))
results_golden = list(ranks_from_sequence(final))

тут почему-то ничего посчитать не получилось :(

In [20]:
print('%0.1f' % spearman_correlation(
  ranks_from_sequence(results_pmi),
  ranks_from_sequence(results_golden)))

0.0


In [22]:
print('%0.1f' % spearman_correlation(
  ranks_from_sequence(results_dice),
  ranks_from_sequence(results_golden)))

0.0


In [23]:
print('%0.1f' % spearman_correlation(
  ranks_from_sequence(results_lklh),
  ranks_from_sequence(results_golden)))

0.0
